In [19]:
# import relevant libraries
import numpy as np

In [20]:
class Area:
    def __init__(self, area_id, name):
        self.area_id = area_id
        self.name = name
        self.priority_weight = 0
        self.sub_areas = []

    def add_sub_area(self, sub_area):
        self.sub_areas.append(sub_area)
        self._update_priority_weight()

    def _update_priority_weight(self):
        if self.sub_areas:
            self.priority_weight = np.average([o.priority_weight for o in self.sub_areas])

In [21]:
class SubArea:
    def __init__(self, sub_area_id):
        self.sub_area_id = sub_area_id
        self.buildings = [] 
        self.area = None
        self.priority_weight = 0
        self.sub_team = None
        self.average_occupancy = None

    def add_building(self, building):
        self.buildings.append(building)
        building.sub_area = self
        building.area = self.area
        self._update_priority_weight()
        self.average_occupancy = self._calculate_average_occupancy()

    def _update_priority_weight(self):
        if self.buildings:
            self.priority_weight = np.average([o.priority_weight for o in self.buildings])

    def _calculate_average_occupancy(self):
        if self.buildings:
            total_occupancy = sum(building.occupancy for building in self.buildings)
            return total_occupancy / len(self.buildings)
        return 0.0

In [29]:
class Building:
    def __init__(self, building_id, geometry, center_point, building_type, building_height, damage_state, occupancy_type, occupancy, injury_severity):
        self.building_id = building_id
        self.geometry = geometry
        self.center_point = center_point
        self.building_type = building_type
        self.building_height = building_height
        self.damage_state = damage_state
        self.occupancy_type = occupancy_type
        self.occupancy = occupancy
        self.injury_severity = injury_severity
        self.priority_weight = 0
        self.area = None
        self.sub_area = None
    
    # create a class method to calculate priority weight for each building in building class
    # this method will be called in the building class

    DS_RATINGS = [0.34, 0.29, 0.22, 0.07, 0.05]
    OCCUPANCY_RATINGS = [0.48, 0.27, 0.13, 0.07, 0.05]
    SEVERITY_RATINGS = [0.49, 0.25, 0.14, 0.05]

    @classmethod
    def calculate_priority_weight(cls, building):
        ds_rating = cls._calculate_ds_rating(building.damage_state)
        rl_rating = cls._calculate_rl_rating(building.damage_state, building.building_type, building.building_height)
        occ_rating = cls._calculate_occ_rating(building.occupancy_type)
        sev_rating = cls._calculate_sev_rating(building.injury_severity)
        trp_rating = cls._calculate_trp_rating(building.occupancy, building.sub_area.average_occupancy)

        building.priority_weight = round(cls._priority_rating(ds_rating, rl_rating, occ_rating, sev_rating, trp_rating),4)

        return building.priority_weight
    
    @staticmethod
    # calculate damage state rating based on damage state list [5, 4, 3, 2, 1]
    def _calculate_ds_rating(damage_state):
        ds_rating = sum(ds * rating for ds, rating in zip(damage_state, Building.DS_RATINGS))
        return ds_rating
    
    @staticmethod
    def _calculate_rl_rating(damage_state, building_type, building_height):
        max_ds3 = max(damage_state[2:])
        if (max_ds3 > 0.6) and ((building_height > 8) or (building_type == "Industrial")):
            return 0.81
        else:
            return 0.19
    
    @staticmethod
    def _calculate_occ_rating(occupancy_type):
        if occupancy_type > 0.8:
            return 0.48
        if occupancy_type > 0.6:
            return 0.27
        if occupancy_type > 0.4:
            return 0.13
        if occupancy_type > 0.2:
            return 0.07
        else:
            return 0.05
        
    @staticmethod
    def _calculate_sev_rating(injury_severity):
        injury_weight = sum(injury_severity)
        overall_rating = sum((severity / injury_weight) * rating for severity, rating in zip(injury_severity, Building.SEVERITY_RATINGS))
        return round(overall_rating, 3)
    
    @staticmethod
    def _calculate_trp_rating(occupancy, average_occupancy):
        if average_occupancy is not None and (occupancy > average_occupancy):
            return 0.7
        else:
            return 0.3
        
    @staticmethod
    def _priority_rating(ds_rating, rl_rating, occ_rating, sev_rating, trp_rating):
        return (0.25 * ds_rating) + (0.2 * rl_rating) + (0.1 * occ_rating) + (0.25 * sev_rating) + (0.2 * trp_rating)

In [37]:
import random  # Ensure the import statement is at the top of your script

# Create area objects
area1 = Area(1, "Area 1")

# Create sub area objects
sub_area1 = SubArea(1)
sub_area2 = SubArea(2)
sub_area3 = SubArea(3)

# Add sub areas to the area
area1.add_sub_area(sub_area1)
area1.add_sub_area(sub_area2)
area1.add_sub_area(sub_area3)

# Create building objects
# For loop to create 5 buildings in each sub area
for i in range(5):
    building = Building(
        building_id=i,
        geometry="square",
        center_point="center",
        building_type=random.choice(["Residential", "Commercial", "Industrial"]),
        building_height=random.uniform(4, 20),
        damage_state=[random.uniform(0, 1) for _ in range(5)],
        occupancy_type=random.uniform(0, 1),
        occupancy=random.randint(10, 100),
        injury_severity=[random.uniform(0, 1) for _ in range(4)]
    )
    sub_area1.add_building(building)

    building = Building(
        building_id=i,
        geometry="square",
        center_point="center",
        building_type=random.choice(["Residential", "Commercial", "Industrial"]),
        building_height=random.uniform(4, 20),
        damage_state=[random.uniform(0, 1) for _ in range(5)],
        occupancy_type=random.uniform(0, 1),
        occupancy=random.randint(10, 100),
        injury_severity=[random.uniform(0, 1) for _ in range(4)]
    )
    sub_area2.add_building(building)

    building = Building(
        building_id=i,
        geometry="square",
        center_point="center",
        building_type=random.choice(["Residential", "Commercial", "Industrial"]),
        building_height=random.uniform(4, 20),
        damage_state=[random.uniform(0, 1) for _ in range(5)],
        occupancy_type=random.uniform(0, 1),
        occupancy=random.randint(10, 100),
        injury_severity=[random.uniform(0, 1) for _ in range(4)]
    )
    sub_area3.add_building(building)


# calculate priority weight for each building and sub area
for building in sub_area1.buildings:
    Building.calculate_priority_weight(building)
for building in sub_area2.buildings:
    Building.calculate_priority_weight(building)
for building in sub_area3.buildings:
    Building.calculate_priority_weight(building)
sub_area1._update_priority_weight()
sub_area2._update_priority_weight()
sub_area3._update_priority_weight()

# print priority weight for each building and sub area
for building in sub_area1.buildings:
    print(building.priority_weight) 
# for building in sub_area2.buildings:
#     print(building.priority_weight)
print(" ")
for building in sub_area3.buildings:
    print(building.priority_weight)
print(" ")
print(sub_area1.priority_weight)
# print(sub_area2.priority_weight)
print(" ")
print(sub_area3.priority_weight)




0.437
0.4196
0.3192
0.5073
0.4282
 
0.3169
0.557
0.4382
0.4437
0.3242
 
0.42225999999999997
 
0.41600000000000004
